<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/006_2_calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q python-dateutil
!pip install -q jpholiday
!apt-get install -y poppler-utils
!pip install -q weasyprint pdf2image

In [ ]:
# @title カレンダー表示（複数月表示用）

from weasyprint import HTML as WeasyHTML
from weasyprint.text.fonts import FontConfiguration
from pdf2image import convert_from_path
from IPython.display import display, HTML
import os
import datetime
import calendar
import jpholiday
import ipywidgets as widgets
from google.colab import files

# ▼HTML生成（複数月分）
def generate_full_calendar_html(year, start_month, count):
    cal = calendar.Calendar(firstweekday=6)
    today = datetime.date.today()

    html = "<style>td, th { padding:4px; border:1px solid #999; text-align:center; }</style>"
    for i in range(count):
        y = year + (start_month + i - 1) // 12
        m = (start_month + i - 1) % 12 + 1
        html += f"<h4>{y}年 {m}月</h4>"
        html += "<table><tr>" + "".join([f"<th>{d}</th>" for d in ["日", "月", "火", "水", "木", "金", "土"]]) + "</tr>"

        for week in cal.monthdatescalendar(y, m):
            html += "<tr>"
            for day in week:
                is_today = (day == today)
                is_other_month = (day.month != m)
                is_holiday = jpholiday.is_holiday(day)

                style = ""
                if is_other_month:
                    style += "color: #aaa;"
                elif is_today:
                    style += "background-color: #ffeb3b;"
                elif is_holiday or day.weekday() >= 5:
                    style += "color: red;"

                tooltip = f"title='{jpholiday.is_holiday_name(day)}'" if is_holiday else ""
                html += f"<td style='{style}' {tooltip}>{day.day}</td>"
            html += "</tr>"
        html += "</table><br>"

    return html

# ▼保存処理
def save_calendar_as_image_pdf(year, start_month, count):
    html = generate_full_calendar_html(year, start_month, count)
    filename_base = f"calendar_{year}_{start_month}_{count}months"

    os.makedirs("calendar_output", exist_ok=True)
    html_path = f"calendar_output/{filename_base}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html)

    # PDF出力
    pdf_path = f"calendar_output/{filename_base}.pdf"
    font_config = FontConfiguration()
    WeasyHTML(string=html).write_pdf(pdf_path, font_config=font_config)

    # PNG出力（PDFから画像へ）
    png_path = f"calendar_output/{filename_base}.png"
    images = convert_from_path(pdf_path)
    images[0].save(png_path, 'PNG')  # 最初のページのみ保存（カレンダーは1ページのはず）

    print("✅ PDFとPNGを保存しました！")
    files.download(pdf_path)
    files.download(png_path)

# ▼UIウィジェット
year_input = widgets.IntText(description="年:", value=datetime.date.today().year)
month_input = widgets.IntSlider(description="開始月:", value=datetime.date.today().month, min=1, max=12)
count_input = widgets.IntSlider(description="表示月数:", value=3, min=1, max=12)
save_button = widgets.Button(description="PNG & PDF 保存", button_style='success')
save_output = widgets.Output()

def on_save_clicked(b):
    with save_output:
        save_output.clear_output()
        save_calendar_as_image_pdf(year_input.value, month_input.value, count_input.value)

save_button.on_click(on_save_clicked)

# ▼表示UI
calendar_save_ui = widgets.VBox([
    widgets.HTML("<h3>🖨️ カレンダーをPNG & PDFで保存</h3>"),
    year_input,
    month_input,
    count_input,
    save_button,
    save_output
])
display(calendar_save_ui)